Gathering 

In [ ]:
# Open csv file
import pandas as pd
df_archive=pd.read_csv('twitter-archive-enhanced.csv')
df_archive=df_archive.copy()

In [ ]:
df_archive.head()

In [ ]:
# Open tsv file
import csv

with open('image-predictions.tsv') as tsvfile:
  reader = csv.reader(tsvfile, delimiter='\t')
  for row in reader:
    print(row)

In [ ]:
search_info = pd.read_csv('image-predictions.tsv',delimiter='\t',nrows=20,encoding='utf-8-sig')#skiprows=1000000, 
print(list(search_info.columns.values)) #file header
print(search_info[[search_info.columns[0],search_info.columns[1],search_info.columns[2]]])

In [ ]:
search_info=search_info.copy()

In [ ]:
search_info.head()

In [ ]:
# Export tweet_id from df_archive as a list
tweet_id=df_archive['tweet_id'].tolist()

In [ ]:
tweet_id

In [ ]:
len(tweet_id)


In [ ]:
# Use the tweet_id list to gather data from API
import tweepy

consumer_key = 'wmrQJf8F8LYiJQLUwb79IDxIT'
consumer_secret = 'YnVLTlGZVP9JmSkRkofjzQofWMLu0yYUonlRfjh6yjn5yw33sw'
access_token = '1076983945047220224-5NEF8bbuvnmXyjwIkKEdzCljJGNZuj'
access_secret = 'dY3jcFFVw8OYTqKeSuCA0KH4pc2po2UGJ8ONsxEdjv78a'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

In [ ]:
import time
start= time.time()
print("start")
df_list=[]
for id in tweet_id:
    try:
        tweet = api.get_status(id,tweet_mode='extended')
        text=tweet.full_text
        retweet_count=tweet.retweet_count
        retweeted=tweet.retweet_count
        favorite_count=tweet.favorite_count
        favorited=tweet.favorited
        df_list.append({'id':id, 'full_text':text,'retweet_count':retweet_count,'retweeted':retweeted,
                       'favorite_count':favorite_count,'favorited':favorited})
        print(df_list)
        
    except Exception as e:
        print('id:'+ 'NA')

        end=time.time()
        print(end-start)

In [ ]:
# Save the API data into a dataframe
df_text=pd.DataFrame(df_list)

In [ ]:
df_text.head()

In [ ]:
# Save the dataframe into a csv data file
df_text.to_csv('tweet_text.csv',index=False)

In [ ]:
df_text.info()

In [ ]:
import json

In [ ]:
data=df_list

In [ ]:
# Save API data into a json string file
def obj_dict(obj):
    return obj.__dict__

json_string = json.dumps(df_list, default=obj_dict)

In [ ]:
json_string

In [ ]:
# Save the JSON string file into a txt file
with open('tweet_json.txt','w') as outfile:
    json.dump(json_string,outfile)

In [ ]:
# Open and check the txt file
with open('tweet_json.txt')as json_file:
    data=json.load(json_file)

In [ ]:
df_tweet=pd.read_json(data)
df_tweet=df_tweet.copy()

In [ ]:
df_tweet.head()

In [ ]:
df_tweet.info()

 Assessing data & Cleaning data 

In [ ]:
df_archive.head()

In [ ]:
df_archive.info()

Tidiness issue 1 

Define

Notice there are columns that are not useful for data analysis, and there're columns only have NaN values. I will clean/delete those columns

Code

In [ ]:
df_archive.drop(['in_reply_to_status_id','in_reply_to_user_id','retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp','rating_denominator'],axis=1,inplace=True)

Test

In [ ]:
df_archive.info()

In [ ]:
#  Check if there's duplicated rows
df_archive.duplicated().sum()

In [ ]:
# Check the shape of the dataframe
df_archive.shape

In [ ]:
df_archive.head()

In [ ]:
# Check if there's any null values 
df_archive.isnull().sum()

Quality issue 1 

Define 
Notice the expanded_urls columns have 59 null values, delete this column

code

In [ ]:

df_archive.drop(['expanded_urls'],axis=1,inplace=True)

test

In [ ]:
df_archive.copy()

In [ ]:
# g. Check the statistic for the data frame
df_archive.describe()

Quality issue 2

Define 

Notice the rating_numerator numbers is within a wide range 10-1776, so I'll check
the unique values in this column

code

In [ ]:
df_archive['rating_numerator'].unique()

In [ ]:
# i. Since this columns values runs into a very wide range, categorize these values
# into different levels, so that it can be easily analysed later

In [ ]:
# Based on the statisticds , create Bin edges that will be used to "cut" the data into groups
bin_rating_edges=[0,10,11,12,1780]
bin_rating_names=['0-10','11','12','>12']
df_archive['rating_levl'] = pd.cut(df_archive['rating_numerator'], bin_rating_edges, labels=bin_rating_names)

test

In [ ]:
df_archive.head()

Tidiness issue 2

Define

Notice there're 4 columns for "dog stage " information, make them into one column "stage"

Code

In [ ]:
df_archive['stage'] = 'doggo'
df_archive.loc[df_archive['floofer'] == 'floofer', 'stage'] = 'floofer'
df_archive.loc[df_archive['pupper'] =='pupper', 'stage'] = 'pupper'
df_archive.loc[df_archive['puppo']=='puppo','stage']='puppo'

In [ ]:
# k. Drop the useless columns 
df_archive.drop(['timestamp','source','text','rating_numerator','name','doggo','floofer','pupper','puppo'],axis=1,inplace=True)

Test

In [ ]:
df_archive.head()

In [ ]:
# open json file and show it as a dataframe 
import json
with open('tweet_json.txt')as json_file:
    data=json.load(json_file) 
df_tweet=pd.read_json(data)
df_tweet=df_tweet.copy()
df_tweet.head()

In [ ]:
# Get info
df_tweet.info()

In [ ]:
# Check null
df_tweet.isnull().sum()

In [ ]:
df_tweet.duplicated().sum()

In [ ]:
# Get statistics
df_tweet.describe()

Quality issue 3 

Define 

After checked the statistic for the data frame, I noticed the favorite_count and retweet_count are within a very wide range of numbers , and they’re not evenly distributed, so I categorize these values into different levels based on the statistics, and created a number categorized data columns.

Code

In [ ]:
# Based on the statistic, create category for favorite_count and retweet_count
# Bin edges that will be used to "cut" the data into groups
bin_favorite_edges=[0,5500,11100,20000,141000]
bin_favorite_names=['0-5000','5000-11100','11100-20000','20000-141000']
bin_retweet_edges=[0,2000,3500,6000,62000]
bin_retweet_names=['0-2000','2000-3500','3500-6000','6000-62000']

In [ ]:
# create columns that categorize retweet_count and favorite_count
df_tweet['favor_count'] = pd.cut(df_tweet['favorite_count'], bin_favorite_edges,labels=bin_favorite_names)

In [ ]:
df_tweet['retwe_count']=pd.cut(df_tweet['retweet_count'], bin_retweet_edges, labels=bin_retweet_names)

Test

In [ ]:
df_tweet.head()

In [ ]:
# Check nulls
df_tweet.isnull().sum()

Quality issue 4 

Define

There are some null_values in favor_count, drop the null_value rows.

Code

In [ ]:
# Drop nulls 
df_tweet.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)

Test

In [ ]:
df_tweet.isnull().sum()

Quality issue 5

Define

There are some duplicated columns like the retweet_count and retweeted is the same column, dropped retweeted columns.

Code

In [ ]:
# Drop duplicated columns
df_tweet.drop(['favorited','retweeted'],axis=1, inplace=True)

Test

In [ ]:
df_tweet.head()

Quality issue 6

Define 

Noticed the ‘full_text’ column is not going to be used for analysis, delete the column.

Code

In [ ]:
df_tweet.drop(['full_text'],axis=1, inplace=True)

Quality issue 7

Define 

In order to merge the tweet_json and twitter-archive-enhaced data later, I changed the ‘id’ to ‘tweet_id’ to match the same column name.

Code

In [ ]:
# rename column so that the data frames can merge later 
df_tweet.rename(index=str,columns={'id':'tweet_id'},inplace=True)

Test

In [ ]:
df_tweet.head()

In [ ]:
# merge dataframes and create a new dataframe df_new
frames=[df_tweet,df_archive]
df_new=df_tweet.merge(df_archive)

In [ ]:
df_new.head()

In [ ]:
df_new.info()

Quality issue 8 

Define 

After merged the data, noticed there’re null values , so I removed the null_value rows.

Code 

In [ ]:
df_tweet.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)

Test

In [ ]:
df_new.isnull().sum()

In [ ]:
df_new.describe()

In [ ]:
df_new.duplicated().sum()

In [ ]:
# Save into a tidy_master file
df_new.to_csv('twitter_archive_master.csv') 

Data Visualization

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')

In [ ]:
df_new=pd.read_csv('twitter_archive_master.csv')

In [ ]:
df_new=df_new.copy()

In [ ]:
# a. get average favorite_count for each stage
favc_rating = df_new.groupby(['stage']).mean()['favorite_count']
favc_rating.sort_values

In [ ]:
# b. get average retweet_count for each stage
retc_rating = df_new.groupby(['stage']).mean()['retweet_count']
retc_rating.sort_values()

In [ ]:
ind = np.arange(len(favc_rating))  # the x locations for the groups
width = 0.35       # the width of the bars

In [ ]:
#c.  plot bars for dog ratings by stage
blue_bars = plt.bar(ind, favc_rating.sort_values(), width, color='b', alpha=.7, label='dog rating')
# title and labels
plt.ylabel('dog favorite ratings')
plt.xlabel('dog stage')
plt.title('Average Dog Favorite Ratings by Stage')
locations = ind + width / 2  # xtick locations
labels = ['doggo', 'pupper', 'floofer', 'puppo']  # xtick labels
plt.xticks(locations, labels)

# legend
plt.legend()

In [ ]:
green_bars = plt.bar(ind, retc_rating.sort_values(), width, color='g', alpha=.7, label='favorite count')
# title and labels
plt.ylabel('Dog retweet counts')
plt.xlabel('dog stage')
plt.title('Average retweet count by Stage')
locations = ind + width / 2  # xtick locations
labels = ['doggo', 'pupper', 'floofer','puppo']  # xtick labels
plt.xticks(locations, labels)

# legend
plt.legend()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
np.random.seed(42)

full_data = pd.read_csv('search_info.csv')
full_data=full_data.copy()
sample_data = full_data.sample(10)

In [ ]:
# Calculate the mean of the prediction 1 from the data sample
sample_mean=full_data['p1_conf'].mean()

In [ ]:
# Bootstrapping from the sample set, select 10 sample data each time and simulate for 10000 times.
# And get the mean from the bootstrap 
means=[]
for _ in range(10000):
    bootsample= sample_data.sample(10,replace=True)
    means.append(bootsample.p1_conf.mean())

In [ ]:
# Standard deviation from the bootstrapping means
np.std(means)

In [ ]:
# Observed mean of prediction one is sample_mean=0.5156
# Null hypothesis p1_whole= p1_sample , which the mean of whole prediction 1 is eqalue to the sample data prediction 1. 
# alpha = 5%
# Ho: P1= 0.5156
# H1: P1><0.5156
null_vals=np.random.normal(0.5156,np.std(means),10000)

In [ ]:
#p_1 distribution from the null
plt.hist(null_vals)

In [ ]:
null_vals.mean()

In [ ]:
p_value= (null_vals>sample_mean).mean()+(null_vals<sample_mean).mean()

In [ ]:
p_value

In [ ]:
# p_value =1, we fail to reject the null